In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
# 安裝 selenium 假如沒有的話
!pip install selenium

!pip install pandas

!pip install beautifulsoup4

In [ ]:
pip install --upgrade pip

In [1]:
import requests
import warnings #用於取消音軟體版本而出現的警告
warnings.filterwarnings('ignore')

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [2]:
import getpass

username = input('輸入FB個人帳號 ： ...')

password = getpass.getpass('請輸FB入密碼 ： ')

輸入FB個人帳號 ： 0000001@gmail.com
請輸FB入密碼 ： 0000000


In [5]:
# chrome_options.add_argument() 添加參數
options = webdriver.ChromeOptions()

# options.add_argument("--disable-infobars") 此方法已失效，要改用下列做法
options.add_experimental_option("excludeSwitches", ['enable-automation']);

# 視窗最大化
options.add_argument("--start-maximized")

#規避bug，文檔提到
options.add_argument('--disable-gpu') 

prefs = {
    # 禁用瀏覽器彈窗
    "profile.default_content_setting_values":{
    "notifications" : 2  },
    # 禁用彈出儲存密碼視窗
    "profile.password_manager_enabled": False, "credentials_enable_service": False
}
options.add_experimental_option('prefs',prefs)

# options.add_argument()

In [6]:
driver = webdriver.Chrome( options = options)
driver.get("https://www.facebook.com/")

driver.find_element("id","email").send_keys(username)
driver.find_element("id","pass").send_keys(password)
driver.find_element("name","login").submit()


#driver.find_element_by_id("pass").send_keys(password)
#driver.find_element_by_name("login").submit()

login_to_pause = 3
driver.implicitly_wait(login_to_pause)

In [7]:
fb_dest = "https://www.facebook.com/MilitarySpokesman"

driver.get(fb_dest)    # 填入所欲爬取的 fb 網址

driver.implicitly_wait(login_to_pause)

In [8]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

In [11]:
max_scroll_pages = 5    # 欲爬取的頁面數目，此頁面數字可以調整。

sec_to_pause = 1    # 暫停駐留在一個頁面的時間，以秒為單位。

for repeated_action in range(1, max_scroll_pages + 1):

    soup = BeautifulSoup(driver.page_source, 'lxml')    # 使用 BeautifulSoup() 來解析取回頁面，解釋模式選擇使用 'lxml' 模式。

    all_titles = soup.find_all('div', {'class':'xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs x126k92a'})   # 抓取所有粉絲頁面公告的訊息。

    time.sleep(sec_to_pause)   # 每滾動一次暫停在該頁面數秒鐘，此時間可以調整。

    # Similar to pressing the PgDn button on the keyboard, it can be replaced with Keys.DOWN (down arrow, but this pull-down progress is slow)
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()

    # Scroll down 500 pixels, the mouse position also changes
    #driver.execute_script("window.scrollBy(0, 500)", "")


print("\n總共有", len(all_titles), "個發文分享！\n")

#print(all_titles, '\n')

df = pd.DataFrame(columns = ['發文主題'])

lst = []
for article in all_titles:
    lst.append(article.getText())
    #df = df.append({'發文主題': article.getText()}, ignore_index = True)

#df_extended = pd.DataFrame(lst, columns=['發文主題'])
df = pd.DataFrame(lst, columns=['發文主題'])
# or columns=df.columns if identical columns

# concatenate to original
#df = pd.concat([df, df_extended])

df.index += 1

df

df.style\
    .set_table_styles([ dict(selector = 'th', props = [('text-align', 'center')])])\
    .set_properties(**{'text-align': 'left', 'font-weight': 'bold'}, color = "blue")


總共有 4 個發文分享！



,發文主題
1,中華民國26年7月7日，盧溝橋事變爆發，這個具有歷史意義的日子，昭示了唯有全國軍民團結一心、同心抗敵，方能抵禦侵略、捍衛國家。
2,#飛彈射擊 #操演群像這群人的身影堆疊成幾個字「我們同心，勢必成功」
3,他們即將和頸線最美的黑天鵝相遇，也將沐沐於最奪目的燦爛豔陽，還有比永遠再多一分鐘的，收操的路。
4,天馬｜神弓｜雷霆｜神鷹 是國軍年度 #飛彈射擊操演，這兩天屏東熱力四射，氣溫高丶士氣更高！


In [43]:
driver.close()

## 社團抓取?

In [9]:
driver.get("https://www.facebook.com/groups/182779590286685")

driver.implicitly_wait(login_to_pause)

In [11]:
max_scroll_pages = 3    # 欲爬取的頁面數目，此頁面數字可以調整。

sec_to_pause = 1    # 暫停駐留在一個頁面的時間，以秒為單位。

for repeated_action in range(1, max_scroll_pages + 1):

    soup = BeautifulSoup(driver.page_source, 'lxml')    # 使用 BeautifulSoup() 來解析取回頁面，解釋模式選擇使用 'lxml' 模式。
    
    # 抓取所有粉絲頁面公告的訊息。
    all_titles = soup.find_all('span', {'class':'xdj266r x11i5rnm xat24cr x1mh8g0r x1vvkbs x126k92a'})
    all_titles = soup.find_all('span', {'class':'x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x6prxxf xvq8zen xo1l8bm xzsf02u'})
    
    time.sleep(sec_to_pause)   # 每滾動一次暫停在該頁面數秒鐘，此時間可以調整。

    # Similar to pressing the PgDn button on the keyboard, it can be replaced with Keys.DOWN (down arrow, but this pull-down progress is slow)
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()

    # Scroll down 500 pixels, the mouse position also changes
    #driver.execute_script("window.scrollBy(0, 500)", "")


print("\n總共有", len(all_titles), "個發文分享！\n")

#print(all_titles, '\n')

df = pd.DataFrame(columns = ['發文主題'])

lst = []
for article in all_titles:
    lst.append(article.getText())
    #df = df.append({'發文主題': article.getText()}, ignore_index = True)

#df_extended = pd.DataFrame(lst, columns=['發文主題'])
df = pd.DataFrame(lst, columns=['發文主題'])
# or columns=df.columns if identical columns

# concatenate to original
#df = pd.concat([df, df_extended])

df.index += 1

df

df.style\
    .set_table_styles([ dict(selector = 'th', props = [('text-align', 'center')])])\
    .set_properties(**{'text-align': 'left', 'font-weight': 'bold'}, color = "blue")


總共有 13 個發文分享！



,發文主題
1,#精選フブキ黑上フブキはこの世で1番かわいい白上フブキはこの世で1番かわいい固定半夜發 日更 婆圖庫
2,夏天穿泳裝很正常不用遮得對吧
3,꒰・◡・๑꒱ ꒰・◡・๑꒱♬konkonbeats♬フブキ【G F O A T】フブキ1000万回愛してる----------我是刪除線----------
4,1.https://twitter.com/....../1677559040933761025/photo/1
5,2.https://twitter.com/....../1677588332845334529/photo/1
6,#精選フブキ黑上フブキはこの世で1番かわいい白上フブキはこの世で1番かわいい固定半夜發 日更 婆圖庫
7,以下開放幻想時間X
8,꒰・◡・๑꒱ ꒰・◡・๑꒱♬konkonbeats♬フブキ【G F O A T】フブキ1000万回愛してる----------我是刪除線----------
9,1.https://twitter.com/....../1677243653524426752/photo/1
10,2.https://twitter.com/....../1677230803322040320/photo/1


In [12]:
max_scroll_pages = 1    # 欲爬取的頁面數目，此頁面數字可以調整。

sec_to_pause = 1    # 暫停駐留在一個頁面的時間，以秒為單位。

for repeated_action in range(1, max_scroll_pages + 1):

    soup = BeautifulSoup(driver.page_source, 'lxml')    # 使用 BeautifulSoup() 來解析取回頁面，解釋模式選擇使用 'lxml' 模式。
    
    all_imgs = soup.find_all('img', {'class':'x1ey2m1c xds687c x5yr21d x10l6tqk x17qophe x13vifvy xh8yej3'})
    
    time.sleep(sec_to_pause)   # 每滾動一次暫停在該頁面數秒鐘，此時間可以調整。

    # Similar to pressing the PgDn button on the keyboard, it can be replaced with Keys.DOWN (down arrow, but this pull-down progress is slow)
    ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()

    # Scroll down 500 pixels, the mouse position also changes
    #driver.execute_script("window.scrollBy(0, 500)", "")

imgs = []
for article in all_imgs:
    imgs.append(article['src'])

df = pd.DataFrame(imgs, columns=['圖片連結'])
# or columns=df.columns if identical columns

df.index += 1

df

df.style\
    .set_table_styles([ dict(selector = 'th', props = [('text-align', 'center')])])\
    .set_properties(**{'text-align': 'left', 'font-weight': 'bold'}, color = "blue")

,圖片連結
1,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358609972_2999458820198892_1976632779426752520_n.jpg?stp=dst-jpg_p350x350&_nc_cat=108&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=kKg-uZh_emYAX9FNQBR&_nc_oc=AQmBdwyPuDTffRMwnyEwgOhcQ7516Ei458q62106VUAKVyDJUhj1e5Opwj_J5NUT4UvZsKhVeXOqCct0HKrIQv_o&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfBRacoAitP5lD7END8BmiYugBo7hrX5ek_HwfQRq43beQ&oe=64AE3330
2,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358527599_2999458900198884_6637881073017035328_n.jpg?stp=dst-jpg_s600x600&_nc_cat=108&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=fh8Xcub9ppcAX-6H8On&_nc_oc=AQlSWjKL6RiE9sInYzyg_vqf6oNfZvoAFINULPvBVGA_qVjwO0n-v6TgeHnLEWR_zd27Pk3TW-cAPtYito227tvy&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfDF4S9CDrB59v4_zh9nKRHvUEBX6b-VrtxZclbfBecP7g&oe=64AF777C
3,https://scontent.fkhh1-1.fna.fbcdn.net/v/t39.30808-6/357725089_2999458793532228_8622148602423536212_n.jpg?stp=dst-jpg_s600x600&_nc_cat=106&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=9D5RzWxkSIgAX-JLe8S&_nc_oc=AQkswjUbkWU-T8t8fweIqCuxICJwpi0LpkbpaWXej-0znyLGQ0D6ktwNrigTFLJxCu3hqBSJ0fOXBFEyL6eTQAjT&_nc_ht=scontent.fkhh1-1.fna&oh=00_AfCSygdaMZMoSY5mPF3gRm1lziGqw5yq5-hj32Lhdw6B3A&oe=64AF2E0B
4,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/357736223_2999458860198888_7396194236137656692_n.jpg?stp=dst-jpg_s600x600&_nc_cat=104&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=7wmFFe1FxRAAX9qeIW6&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfCiHKTHDTFSMsgexM9xMoHU0282al60inpVCRnF2odzOg&oe=64AF0FD8
5,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/359376529_2999459066865534_5462554928081380391_n.jpg?stp=dst-jpg_s600x600&_nc_cat=109&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=qoZMDqG5sSYAX8Nl3ih&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfDkgxq9vN0bMPJVsK7v0I5ga-MCNnaI9a7DBkvHi6vM4A&oe=64AF0145
6,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/357725596_6542269052499914_4692525590462970816_n.jpg?stp=dst-jpg_s600x600&_nc_cat=111&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=nt1TMYpewkgAX9ZJus5&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfD_DcApxa230w5d6SAv3sQm4Ay80_MQz0cM2R3DZ6t70g&oe=64AF282A
7,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358106923_6542269312499888_8212348738835891614_n.jpg?stp=dst-jpg_s600x600&_nc_cat=109&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=XguL9k7dxnEAX9igA2x&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfA6U_z5wZIDvn_MqxX0Vwa5DOgwyI_EuX6fUXhu4e8JrA&oe=64AEABAD
8,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358044981_6542269392499880_5355577257480181333_n.jpg?stp=dst-jpg_s600x600&_nc_cat=111&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=8xLj9QnbutYAX-9qzF_&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfCS1Q7GPciRmxqnYa2mAz2jcYWYZIvbJZ0rQrc8jVxh-g&oe=64B00093
9,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358417959_6542269479166538_164546661644184777_n.jpg?stp=dst-jpg_s600x600&_nc_cat=107&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=TPgOkV9F31EAX-pX6FG&_nc_oc=AQmuN79Q0vYEpnXZxEyQBtlITDBq7Bfi0p_VQKQ5yK21iWxUwbQBu9LFXBPvn05Jz5mSsJ2By3xCMfGHXwZMu9ER&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfA6QN8TaARsWh1drNG0EA8p3sqlXUt57GXr7nsYAhwBgA&oe=64AEED98
10,https://scontent.fkhh1-2.fna.fbcdn.net/v/t39.30808-6/358119409_6542269542499865_8492258734975398143_n.jpg?stp=dst-jpg_s600x600&_nc_cat=110&ccb=1-7&_nc_sid=5cd70e&_nc_ohc=m58rc13dEvgAX93HeFr&_nc_ht=scontent.fkhh1-2.fna&oh=00_AfC847J2mIxqkR6CWLe6peZMJ0VM3LkRaMjvqgWZGm2NEA&oe=64AEA3B0


In [34]:
import requests
img_data = requests.get(imgs[0]).content 

with open("netflix.jpg", "wb") as handler: 
    handler.write(img_data) 

driver.close()

In [13]:
import os

for index, link in enumerate(imgs):
    if not os.path.exists("images"):
        os.mkdir("images")  # 建立資料夾
 
    img = requests.get(link)  # 下載圖片
 
    with open("images\\" + "fbk_club0{}.jpg".format(index+1), "wb") as file:  # 開啟資料夾及命名圖片檔
        file.write(img.content)  # 寫入圖片的二進位碼

#原本寫法        
#imgs_data = []

#for i in range(len(imgs)):  

#    print(i)
    
#    #imgs_data[i].append(requests.get(imgs[i]).content)
#    img_wb = requests.get(imgs[i])
#    with open("test{}.jpg".format(i+1), "wb") as handler: 
        
#        handler.write(img_wb.content) 

## 爬蟲 巴哈姆特電玩資訊網站

### 已知問題
1. 使用selenium webdriver開啟的登入網頁會出現"我不是機器人"的問題
2. 手邊沒資源也沒有技術來破解"我不是機器人"登入問題
(改用手動登入後，將cookie給記錄下來後，再重新添加cookie至driver，即可解決(?)
### 未確定問題
1. 巴哈姆特登入的cookie的時效多久(可能只有幾小時*)

In [67]:
options.add_argument("User-Agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
                     AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

In [68]:
## 登入帳號密碼

useracc = input("輸入巴哈姆特帳號：")

userpass = getpass.getpass("輸入密碼")

輸入巴哈姆特帳號：sdd989888
輸入密碼········


In [69]:
import json

login_to_pause = 3
bahamute_login = "https://user.gamer.com.tw/login.php"

driver = webdriver.Chrome(options = options)
driver.get(bahamute_login)

driver.find_element('name', "userid").send_keys(useracc)
driver.find_element('name', "password").send_keys(userpass)
# driver.find_element('id', "btn-login").submit()

driver.implicitly_wait(login_to_pause)
time.sleep(30)

cookies = driver.get_cookies()
# print(cookies)

with open("cookies_bahamute.json", "w") as f:
    f.write(json.dumps(cookies))


In [70]:
bahamute = "https://www.gamer.com.tw/"

# 紀錄cookies後再次登入
driver = webdriver.Chrome(options = options)
driver.get(bahamute)
driver.delete_all_cookies()

with open("cookies_bahamute.json", 'r') as of:
    cookie_list = json.loads(of.read())

# 逐行添加cookie
for cookie in cookie_list:
    driver.add_cookie(cookie)

# 重新整理後會發現已經登入了
driver.refresh()

#driver.find_element('name', "userid").send_keys(useracc)
#driver.find_element('name', "password").send_keys(userpass)
#driver.find_element('id', "btn-login").submit()

driver.implicitly_wait(login_to_pause)

In [154]:
# page=1、2、3...依此類推
bahamute = "https://forum.gamer.com.tw/B.php?page=1&bsn=60076"

print("抓取內容中...")

driver.get(bahamute)
soup = BeautifulSoup(driver.page_source, 'lxml')

title_propaganda = soup.find_all("a",{"class" : "b-list__main__title"})
title_main = soup.find_all("p",{"class" : "b-list__main__title"})
master = soup.find_all("p", {"class" : "b-list__count__user"})


# 無法理解...加上這一行之後無法透過forloop把bs4.element.Result的title_propaganda改成titles的bs4.element.Tag的類型
# https://stackoverflow.com/questions/72434442/how-to-fix-attributeerror-resultset-object-has-no-attribute-get-text
# title_propaganda.append(title_main) 


print("提取標籤內容中...")
title_ls = []
for titles in title_propaganda:
    title_ls.append(titles.get_text())
    
for titles in title_main:
    title_ls.append(titles.get_text())


master_ls = []
for masters in master:
    master_ls.append(masters.get_text())



print("以提取所有內容")

df_baha = pd.DataFrame(masters)

抓取內容中...
提取標籤內容中...
以提取所有內容


In [128]:
type(titles)

bs4.element.Tag

In [155]:
# 去掉master_ls[]中多餘的"\n"
for i in range(len(master_ls)):
    master_ls[i] = master_ls[i].replace("\n","")

master_ls

['johnny860726',
 'edfrmpc44ic',
 'henry72122',
 'a29657016',
 'rainnnn27',
 'kuroko27169',
 'jeff81049',
 'firejp6',
 'bigmac108',
 'oska243',
 'beautifully',
 'jkl098178201',
 'vm0vm4',
 'cutarlen1223',
 'cd8008012115',
 'hugefire4444',
 'm122306835',
 'john95173',
 'catporn',
 'HoshizoraCat',
 'CCHAHA7788',
 'zero77716',
 'p1048576',
 'abcd8820',
 'Skycat119',
 'FCAK2895',
 'as36865231',
 'vitamilk',
 'howie123',
 'dafeichai']

In [156]:
title_ls

['【分流】板建區 / 進板圖招募 / 現任小組名單',
 '【宣導】兒少性剝削修正法案宣導',
 '【問題】【認真】新竹協尋 玄鳳黑牛鸚鵡',
 '【認真】楊梅區武營街與梅山西街市場路口，協尋行車記錄器',
 '【問題】【認真】雲林協尋 白底橘虎斑貓',
 '【問題】【認真】萬華協尋 褐色虎斑貓',
 '【問題】場外人讀大學的目的是什麼？',
 '場外魔物獵人串  破曉  6/8 破曉最後更新 原初爵銀龍',
 '【問題】是不是幾乎沒有人討厭咖哩?',
 '新學期的開始我們先來做自我介紹吧',
 '【討論】一人一句名言，其他人猜人物',
 '【閒聊】馬斯克跟祖克柏格鬥 怎麼想都是馬斯克贏阿',
 '【問題】大家玩遊戲都喜歡甚麼職業呢',
 '【閒聊】你們在DC群最常用的貼圖是什麼',
 '【情報】獨家》「鐵門生鏽」房東求償2.5萬 房客控「自然現象」',
 '【心得】張家航的call in超好笑的',
 '【閒聊】巴哈市集旁邊捷入口有一段超臭',
 '【閒聊】雖然已經過氣了，但有沒有人想問跟槍彈辯駁有關的問題?',
 '【閒聊】馬德這裡也太多皇民了',
 '512首動漫歌曲殘酷二選一',
 '【閒聊】在餐廳吃飯，看到有漂亮小姊姊笑著向我走來',
 '【問題】今天台北地下街484特別臭？',
 '【情報】山猴子的一生',
 '【閒聊】一天一張追劇的截圖',
 '［第488天］天佑烏克蘭｜反攻進行式 & 普里格津被起訴和「失蹤」',
 '【心得】好想射精喔，可是熱到沒心情尻槍',
 '【東方】偷偷地…檸檬東方Project串',
 '【討論】台灣IKEA刪除諧音圖「逮八人」的facebook貼文',
 '【討論】統神已離婚機率高達87% (003買麥當勞事件)',
 '【情報】本季的妙廟美少女有料喔!']

In [65]:
html_t = """<html><head><title>Hello World</title></head>
            <body><h2>Test Header</h2><p class="b-list__count__user"><a href="aaaa" target="_blank">ejiejicj8</a></p>
            </body></html>"""

soup = BeautifulSoup(html_t, 'lxml')

master = soup.find_all("p", {"class" : "b-list__count__user"})

for masters in master:
    print("以提取內容")
masters.getText()

以提取內容


'ejiejicj8'

In [183]:
df_master_ls = pd.Da taFrame(master_ls)
df_title_ls = pd.DataFrame(title_ls)

df_baha = pd.concat([df_master_ls, df_title_ls], ignore_index=True, axis = 1)

df_baha.columns = ["作者", "發文標題"]

df_baha.index += 1

df_baha

df_baha.style\
    .set_table_styles([ dict(selector = 'th', props = [('text-align', 'center')])])\
    .set_properties(**{'text-align': 'left', 'font-weight': 'bold'})

,作者,發文標題
1,johnny860726,【分流】板建區 / 進板圖招募 / 現任小組名單
2,edfrmpc44ic,【宣導】兒少性剝削修正法案宣導
3,henry72122,【問題】【認真】新竹協尋 玄鳳黑牛鸚鵡
4,a29657016,【認真】楊梅區武營街與梅山西街市場路口，協尋行車記錄器
5,rainnnn27,【問題】【認真】雲林協尋 白底橘虎斑貓
6,kuroko27169,【問題】【認真】萬華協尋 褐色虎斑貓
7,jeff81049,【問題】場外人讀大學的目的是什麼？
8,firejp6,場外魔物獵人串 破曉 6/8 破曉最後更新 原初爵銀龍
9,bigmac108,【問題】是不是幾乎沒有人討厭咖哩?
10,oska243,新學期的開始我們先來做自我介紹吧


In [172]:
pd.__version__

'2.0.3'

In [169]:
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.199)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00C2A813+48355]
	(No symbol) [0x00BBC4B1]
	(No symbol) [0x00AC5358]
	(No symbol) [0x00AAD293]
	(No symbol) [0x00B0E37B]
	(No symbol) [0x00B12842]
	(No symbol) [0x00B0A703]
	(No symbol) [0x00AE82DC]
	(No symbol) [0x00AE93DD]
	GetHandleVerifier [0x00E8AABD+2539405]
	GetHandleVerifier [0x00ECA78F+2800735]
	GetHandleVerifier [0x00EC456C+2775612]
	GetHandleVerifier [0x00CB51E0+616112]
	(No symbol) [0x00BC5F8C]
	(No symbol) [0x00BC2328]
	(No symbol) [0x00BC240B]
	(No symbol) [0x00BB4FF7]
	BaseThreadInitThunk [0x767400C9+25]
	RtlGetAppContainerNamedObjectPath [0x76EF7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x76EF7B1E+238]
